# I hate ML :)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip uninstall -y gensim numpy scipy
!pip install numpy==1.24.3 scipy==1.10.1 gensim==4.3.1

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scipy 1.14.1
Uninstalling scipy-1.14.1:
  Successfully uninstalled scipy-1.14.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 44.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11.4, but you have scipy 1.10.1 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud,STOPWORDS
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, classification_report, confusion_matrix
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier

In [2]:
file_path = "/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/train.csv"

df = pd.read_csv(file_path)

In [3]:
df['total_text'] = df['TITLE'] + ' ' + df['ABSTRACT']
df.set_index('ID', inplace=True)
df = df[['total_text', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']]
df['labels'] = df.apply(lambda x: [x['Computer Science'], x['Physics'], x['Mathematics'], x['Statistics'], x['Quantitative Biology'], x['Quantitative Finance']], axis=1)

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')
stop = stopwords.words('english')

def word_lemmatizer(text):
    return [WordNetLemmatizer().lemmatize(i) for i in text]

def clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_tokenize(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_lemmatizer(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join(x))
    return df

train_clean_df = clean_text(df, 'total_text', 'total_text_clean')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
train_clean_df = train_clean_df[['total_text_clean','labels']]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(train_clean_df['total_text_clean'])
print(tfidf_matrix.shape)

(20972, 226267)


In [7]:
import os
ensemble_dir = "/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/ensemble"
os.makedirs(ensemble_dir, exist_ok=True)
tfidf_file_path = os.path.join(ensemble_dir, "tfidf_matrix.npz")
from scipy.sparse import save_npz
save_npz(tfidf_file_path, tfidf_matrix)

import pickle
vectorizer_file_path = os.path.join(ensemble_dir, "tfidf_vectorizer.pkl")
with open(vectorizer_file_path, 'wb') as f:
    pickle.dump(vectorizer, f)

# Word2Vec


In [7]:
texts = train_clean_df["total_text_clean"].apply(lambda x: x.split())
labels = np.array(train_clean_df["labels"].tolist())

w2v_model = Word2Vec(sentences=texts, vector_size=50, window=5, min_count=1)
def sentence_to_vector(sentence, model):
    vectors = [model.wv[word] for word in sentence if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

X = np.array([sentence_to_vector(sent, w2v_model) for sent in texts])
y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# prompt: save word2vec as pikl

import pickle
import os

# Specify the directory and filename for saving the Word2Vec model
w2v_model_path = "word2vec_model.pkl"

# Save the Word2Vec model using pickle
with open(w2v_model_path, 'wb') as f:
    pickle.dump(w2v_model, f)


In [ ]:
# prompt: save the word2vec

import os
from gensim.models import Word2Vec

# Specify the directory and filename for saving the model
model_dir = "/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/ensemble"
os.makedirs(model_dir, exist_ok=True)  # Create the directory if it doesn't exist
model_file_path = os.path.join(model_dir, "word2vec_model")

# Save the Word2Vec model
w2v_model.save(model_file_path)


In [10]:
word_vectors = {word: w2v_model.wv[word] for word in w2v_model.wv.index_to_key}
vector_size = w2v_model.vector_size
np.save('word_vectors.npy', (word_vectors, vector_size))

# DT Bagging Base

In [15]:
base_estimator = DecisionTreeClassifier(max_depth=8)

bagging_model = BaggingClassifier(estimator=base_estimator, n_estimators=10, random_state=42)
multi_output_model_bagging = MultiOutputClassifier(estimator=bagging_model)
multi_output_model_bagging.fit(X_train, y_train)
y_pred = multi_output_model_bagging.predict(X_test)
train_pred = multi_output_model_bagging.predict(X_train)
test_pred = multi_output_model_bagging.predict(X_test)

train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

Train Accuracy: 0.718781665375216
Test Accuracy: 0.6243146603098927


In [18]:
print("Classification Metrics:")
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Precision (micro):", precision_score(y_test, test_pred, average='micro'))
print("Recall (micro):", recall_score(y_test, test_pred, average='micro'))
print("F1 Score (micro):", f1_score(y_test, test_pred, average='micro'))

y_prob = multi_output_model_bagging.predict_proba(X_test)
y_prob = np.array([[cls[1] for cls in sample] for sample in y_prob])
y_prob = np.swapaxes(y_prob, 0, 1)

print("ROC AUC (macro):", roc_auc_score(y_test, y_prob, average='macro', multi_class='ovr'))
print("Log Loss:", log_loss(y_test, y_prob))
print("\nClassification Report:\n", classification_report(y_test, test_pred, zero_division=0))

print("\nConfusion Matrices (per class):")
for i in range(y_test.shape[1]):
    print(f"Class {i+1}:\n", confusion_matrix(y_test[:, i], test_pred[:, i]))


Classification Metrics:
Accuracy: 0.6243146603098927
Precision (micro): 0.8136112814224402
Recall (micro): 0.7505656108597285
F1 Score (micro): 0.7808178876140042
ROC AUC (macro): 0.9035390662361983
Log Loss: 0.7654764530719498

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.86      0.81      1692
           1       0.90      0.79      0.84      1226
           2       0.84      0.73      0.78      1150
           3       0.78      0.68      0.73      1069
           4       0.39      0.06      0.10       122
           5       0.00      0.00      0.00        45

   micro avg       0.81      0.75      0.78      5304
   macro avg       0.61      0.52      0.54      5304
weighted avg       0.80      0.75      0.77      5304
 samples avg       0.80      0.78      0.77      5304


Confusion Matrices (per class):
Class 1:
 [[2081  422]
 [ 243 1449]]
Class 2:
 [[2861  108]
 [ 262  964]]
Class 3:
 [[2883  162]
 [ 316  834]]
Class 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [19]:
model_file_path = "/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/ensemble/multi_output_model_bagging.pkl"
with open(model_file_path, 'wb') as f:
    pickle.dump(multi_output_model_bagging, f)

# DT Bagging Hyperparameter


In [24]:
base_tree = DecisionTreeClassifier()
bagging_model = BaggingClassifier(estimator=base_tree, random_state=22)
multioutput_bagging_model = MultiOutputClassifier(bagging_model)
param_grid = {
    'estimator__n_estimators': [5, 20],
    'estimator__max_samples': [0.5, 1.0],
    'estimator__estimator__max_depth': [3, 5]
}
grid_search_cv = GridSearchCV(
    estimator=multioutput_bagging_model,
    param_grid=param_grid,
    cv=3,
    scoring='f1_micro'
)
grid_search_cv.fit(X_train, y_train)
best_multioutput_model = grid_search_cv.best_estimator_
test_pred = best_multioutput_model.predict(X_test)
train_pred = best_multioutput_model.predict(X_train)

train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

Train Accuracy: 0.6299696012397926
Test Accuracy: 0.6104886769964243


In [25]:
print("Classification Metrics:")
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Precision (micro):", precision_score(y_test, test_pred, average='micro'))
print("Recall (micro):", recall_score(y_test, test_pred, average='micro'))
print("F1 Score (micro):", f1_score(y_test, test_pred, average='micro'))

y_prob = best_multioutput_model.predict_proba(X_test)
y_prob = np.array([[cls[1] for cls in sample] for sample in y_prob])
y_prob = np.swapaxes(y_prob, 0, 1)

print("ROC AUC (macro):", roc_auc_score(y_test, y_prob, average='macro', multi_class='ovr'))
print("Log Loss:", log_loss(y_test, y_prob))
print("\nClassification Report:\n", classification_report(y_test, test_pred, zero_division=0))

print("\nConfusion Matrices (per class):")
for i in range(y_test.shape[1]):
    print(f"Class {i+1}:\n", confusion_matrix(y_test[:, i], test_pred[:, i]))


Classification Metrics:
Accuracy: 0.6104886769964243
Precision (micro): 0.8118357991248176
Recall (micro): 0.7345399698340875
F1 Score (micro): 0.7712560625556766
ROC AUC (macro): 0.8952790574354338
Log Loss: 0.7847349522116038

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.84      0.80      1692
           1       0.90      0.77      0.83      1226
           2       0.83      0.72      0.78      1150
           3       0.77      0.66      0.71      1069
           4       0.50      0.01      0.02       122
           5       0.00      0.00      0.00        45

   micro avg       0.81      0.73      0.77      5304
   macro avg       0.63      0.50      0.52      5304
weighted avg       0.80      0.73      0.76      5304
 samples avg       0.78      0.77      0.76      5304


Confusion Matrices (per class):
Class 1:
 [[2080  423]
 [ 274 1418]]
Class 2:
 [[2867  102]
 [ 284  942]]
Class 3:
 [[2880  165]
 [ 318  832]]
Class 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [27]:
model_file_path = "/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/ensemble/multi_output_model_bagging_best.pkl"
with open(model_file_path, 'wb') as f:
    pickle.dump(multi_output_model_bagging, f)


# SVM Bagging Base

In [28]:
base_estimator = SVC(probability=True, kernel='rbf', C=1.0)
bagging_model = BaggingClassifier(estimator=base_estimator, n_estimators=5, random_state=42)
multi_output_model_bagging = MultiOutputClassifier(estimator=bagging_model)
multi_output_model_bagging.fit(X_train, y_train)

train_pred = multi_output_model_bagging.predict(X_train)
test_pred = multi_output_model_bagging.predict(X_test)

train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")


Train Accuracy: 0.6517255766823628
Test Accuracy: 0.6405244338498212


In [29]:
print("Classification Metrics:")
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Precision (micro):", precision_score(y_test, test_pred, average='micro'))
print("Recall (micro):", recall_score(y_test, test_pred, average='micro'))
print("F1 Score (micro):", f1_score(y_test, test_pred, average='micro'))

y_prob = multi_output_model_bagging.predict_proba(X_test)
y_prob = np.array([[cls[1] for cls in sample] for sample in y_prob])
y_prob = np.swapaxes(y_prob, 0, 1)

print("ROC AUC (macro):", roc_auc_score(y_test, y_prob, average='macro', multi_class='ovr'))
print("Log Loss:", log_loss(y_test, y_prob))
print("\nClassification Report:\n", classification_report(y_test, test_pred, zero_division=0))

print("\nConfusion Matrices (per class):")
for i in range(y_test.shape[1]):
    print(f"Class {i+1}:\n", confusion_matrix(y_test[:, i], test_pred[:, i]))


Classification Metrics:
Accuracy: 0.6405244338498212
Precision (micro): 0.8319170984455958
Recall (micro): 0.7567873303167421
F1 Score (micro): 0.7925757725343074
ROC AUC (macro): 0.930989723312211
Log Loss: 0.7046020673981761

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.85      0.82      1692
           1       0.90      0.81      0.86      1226
           2       0.86      0.73      0.79      1150
           3       0.79      0.69      0.74      1069
           4       0.40      0.02      0.03       122
           5       1.00      0.02      0.04        45

   micro avg       0.83      0.76      0.79      5304
   macro avg       0.79      0.52      0.55      5304
weighted avg       0.83      0.76      0.78      5304
 samples avg       0.81      0.79      0.78      5304


Confusion Matrices (per class):
Class 1:
 [[2124  379]
 [ 250 1442]]
Class 2:
 [[2863  106]
 [ 229  997]]
Class 3:
 [[2912  133]
 [ 314  836]]
Class 4

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [30]:
model_file_path = "/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/ensemble/multi_output_model_bagging_svm.pkl"
with open(model_file_path, 'wb') as f:
    pickle.dump(multi_output_model_bagging, f)


# SVM Bagging Hyperparameter (havent run)

In [ ]:
base_svm = SVC(probability=True)

bagging_svm_model = BaggingClassifier(estimator=base_svm, random_state=22)

multioutput_bagging_svm = MultiOutputClassifier(bagging_svm_model)

param_grid = {
    'estimator__n_estimators': [5, 10],
    'estimator__max_samples': [0.5, 1.0],
    'estimator__estimator__C': [0.1, 1, 10],
    'estimator__estimator__kernel': ['linear', 'rbf']
}

grid_search_cv = GridSearchCV(
    estimator=multioutput_bagging_svm,
    param_grid=param_grid,
    cv=3,
    scoring='f1_micro'
)
grid_search_cv.fit(X_train, y_train)
best_multioutput_svm_model = grid_search_cv.best_estimator_
test_pred = best_multioutput_svm_model.predict(X_test)
train_pred = best_multioutput_svm_model.predict(X_train)


In [ ]:
print("Classification Metrics:")
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Precision (micro):", precision_score(y_test, test_pred, average='micro'))
print("Recall (micro):", recall_score(y_test, test_pred, average='micro'))
print("F1 Score (micro):", f1_score(y_test, test_pred, average='micro'))

y_prob = best_multioutput_model.predict_proba(X_test)
y_prob = np.array([[cls[1] for cls in sample] for sample in y_prob])
y_prob = np.swapaxes(y_prob, 0, 1)

print("ROC AUC (macro):", roc_auc_score(y_test, y_prob, average='macro', multi_class='ovr'))
print("Log Loss:", log_loss(y_test, y_prob))
print("\nClassification Report:\n", classification_report(y_test, test_pred, zero_division=0))

print("\nConfusion Matrices (per class):")
for i in range(y_test.shape[1]):
    print(f"Class {i+1}:\n", confusion_matrix(y_test[:, i], test_pred[:, i]))


# RF Bagging Base (havent run)

In [ ]:
base_estimator = RandomForestClassifier(random_state=42)
bagging_model = BaggingClassifier(estimator=base_estimator, n_estimators=5, random_state=42)
multi_output_model_bagging = MultiOutputClassifier(estimator=bagging_model)
multi_output_model_bagging.fit(X_train, y_train)

train_pred = multi_output_model_bagging.predict(X_train)
test_pred = multi_output_model_bagging.predict(X_test)

train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:

print("Classification Metrics:")
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Precision (micro):", precision_score(y_test, test_pred, average='micro'))
print("Recall (micro):", recall_score(y_test, test_pred, average='micro'))
print("F1 Score (micro):", f1_score(y_test, test_pred, average='micro'))

y_prob = multi_output_model_bagging.predict_proba(X_test)
y_prob = np.array([[cls[1] for cls in sample] for sample in y_prob])
y_prob = np.swapaxes(y_prob, 0, 1)

print("ROC AUC (macro):", roc_auc_score(y_test, y_prob, average='macro', multi_class='ovr'))
print("Log Loss:", log_loss(y_test, y_prob))
print("\nClassification Report:\n", classification_report(y_test, test_pred, zero_division=0))

print("\nConfusion Matrices (per class):")
for i in range(y_test.shape[1]):
    print(f"Class {i+1}:\n", confusion_matrix(y_test[:, i], test_pred[:, i]))


In [ ]:
model_file_path = "/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/ensemble/multi_output_model_bagging_rf.pkl"
with open(model_file_path, 'wb') as f:
    pickle.dump(multi_output_model_bagging, f)

# SVM Adaboost Base (didnt work)

In [34]:
base_estimator = SVC(probability=True)
ada_boost_model = AdaBoostClassifier(estimator=base_estimator, n_estimators=5, random_state=42, algorithm='SAMME')

multi_output_model_adaboost = MultiOutputClassifier(estimator=ada_boost_model)
multi_output_model_adaboost.fit(X_train, y_train)

train_pred = multi_output_model_adaboost.predict(X_train)
test_pred = multi_output_model_adaboost.predict(X_test)

train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 

Train Accuracy: 0.0
Test Accuracy: 0.0


In [36]:
print("Classification Metrics:")
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Precision (micro):", precision_score(y_test, test_pred, average='micro'))
print("Recall (micro):", recall_score(y_test, test_pred, average='micro'))
print("F1 Score (micro):", f1_score(y_test, test_pred, average='micro'))

y_prob = multi_output_model_adaboost.predict_proba(X_test)
y_prob = np.array([[cls[1] for cls in sample] for sample in y_prob])
y_prob = np.swapaxes(y_prob, 0, 1)

print("ROC AUC (macro):", roc_auc_score(y_test, y_prob, average='macro', multi_class='ovr'))
print("Log Loss:", log_loss(y_test, y_prob))
print("\nClassification Report:\n", classification_report(y_test, test_pred, zero_division=0))

print("\nConfusion Matrices (per class):")
for i in range(y_test.shape[1]):
    print(f"Class {i+1}:\n", confusion_matrix(y_test[:, i], test_pred[:, i]))

Classification Metrics:
Accuracy: 0.0
Precision (micro): 0.0
Recall (micro): 0.0
F1 Score (micro): 0.0


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


KeyboardInterrupt: 

In [ ]:
model_file_path = "/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/ensemble/multi_output_model_adaboost_svm.pkl"
with open(model_file_path, 'wb') as f:
    pickle.dump(multi_output_model_adaboost, f)

# Adaboost

In [42]:
rf_model = MultiOutputClassifier(RandomForestClassifier(n_estimators=5, random_state=22))
rf_model.fit(X_train, y_train)

train_pred = rf_model.predict(X_train)
test_pred = rf_model.predict(X_test)

train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

Train Accuracy: 0.9390236633486321
Test Accuracy: 0.5744934445768772


In [43]:
adaboost_model = MultiOutputClassifier(AdaBoostClassifier(n_estimators=10, random_state=22))
adaboost_model.fit(X_train, y_train)

train_pred = adaboost_model.predict(X_train)
test_pred = adaboost_model.predict(X_test)

train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

Train Accuracy: 0.5441378077129403
Test Accuracy: 0.5463647199046484


In [44]:

print("Classification Metrics:")
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Precision (micro):", precision_score(y_test, test_pred, average='micro'))
print("Recall (micro):", recall_score(y_test, test_pred, average='micro'))
print("F1 Score (micro):", f1_score(y_test, test_pred, average='micro'))

y_prob = adaboost_model.predict_proba(X_test)
y_prob = np.array([[cls[1] for cls in sample] for sample in y_prob])
y_prob = np.swapaxes(y_prob, 0, 1)

print("ROC AUC (macro):", roc_auc_score(y_test, y_prob, average='macro', multi_class='ovr'))
print("Log Loss:", log_loss(y_test, y_prob))
print("\nClassification Report:\n", classification_report(y_test, test_pred, zero_division=0))

print("\nConfusion Matrices (per class):")
for i in range(y_test.shape[1]):
    print(f"Class {i+1}:\n", confusion_matrix(y_test[:, i], test_pred[:, i]))


Classification Metrics:
Accuracy: 0.5463647199046484
Precision (micro): 0.7666531275385865
Recall (micro): 0.7117269984917044
F1 Score (micro): 0.7381697301525225
ROC AUC (macro): 0.8821191844788429
Log Loss: 0.7331947165362701

Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.82      0.77      1692
           1       0.86      0.78      0.82      1226
           2       0.81      0.68      0.74      1150
           3       0.69      0.61      0.65      1069
           4       0.00      0.00      0.00       122
           5       0.00      0.00      0.00        45

   micro avg       0.77      0.71      0.74      5304
   macro avg       0.51      0.48      0.50      5304
weighted avg       0.75      0.71      0.73      5304
 samples avg       0.73      0.74      0.71      5304


Confusion Matrices (per class):
Class 1:
 [[1996  507]
 [ 309 1383]]
Class 2:
 [[2814  155]
 [ 275  951]]
Class 3:
 [[2855  190]
 [ 363  787]]
Class 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [45]:
model_file_path = "/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/ensemble/adaboost_model.pkl"
with open(model_file_path, 'wb') as f:
    pickle.dump(adaboost_model, f)

# Voting Base

In [ ]:
lr = LogisticRegression(max_iter=1000)
dt = DecisionTreeClassifier()
svc = SVC(probability=True)

voting_clf = VotingClassifier(
    estimators=[('lr', lr), ('dt', dt), ('svc', svc)], voting='soft')

multi_voting_clf = MultiOutputClassifier(voting_clf)

multi_voting_clf.fit(X_train, y_train)

test_pred = multi_voting_clf.predict(X_test)
train_pred = multi_voting_clf.predict(X_train)

In [56]:
train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

Train Accuracy: 0.8022292424152113
Test Accuracy: 0.601668653158522


In [57]:
print("Classification Metrics:")
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Precision (micro):", precision_score(y_test, test_pred, average='micro'))
print("Recall (micro):", recall_score(y_test, test_pred, average='micro'))
print("F1 Score (micro):", f1_score(y_test, test_pred, average='micro'))

y_prob = multi_voting_clf.predict_proba(X_test)
y_prob = np.array([[cls[1] for cls in sample] for sample in y_prob])
y_prob = np.swapaxes(y_prob, 0, 1)

print("ROC AUC (macro):", roc_auc_score(y_test, y_prob, average='macro', multi_class='ovr'))
print("Log Loss:", log_loss(y_test, y_prob))
print("\nClassification Report:\n", classification_report(y_test, test_pred, zero_division=0))

print("\nConfusion Matrices (per class):")
for i in range(y_test.shape[1]):
    print(f"Class {i+1}:\n", confusion_matrix(y_test[:, i], test_pred[:, i]))

Classification Metrics:
Accuracy: 0.601668653158522
Precision (micro): 0.8053677525097316
Recall (micro): 0.7411387631975868
F1 Score (micro): 0.7719194894452627
ROC AUC (macro): 0.9243903277712615
Log Loss: 0.7552910457584209

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.82      0.80      1692
           1       0.89      0.81      0.85      1226
           2       0.82      0.73      0.77      1150
           3       0.76      0.66      0.71      1069
           4       0.47      0.06      0.10       122
           5       1.00      0.02      0.04        45

   micro avg       0.81      0.74      0.77      5304
   macro avg       0.78      0.52      0.54      5304
weighted avg       0.80      0.74      0.76      5304
 samples avg       0.79      0.78      0.76      5304


Confusion Matrices (per class):
Class 1:
 [[2102  401]
 [ 310 1382]]
Class 2:
 [[2842  127]
 [ 231  995]]
Class 3:
 [[2856  189]
 [ 309  841]]
Class 4

In [ ]:
model_file_path = "/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/ensemble/votting_classifier_base.pkl"
with open(model_file_path, 'wb') as f:
    pickle.dump(multi_voting_clf, f)


# Voting Classifier Hyperparameter

In [48]:
lr = LogisticRegression(max_iter=1000)
dt = DecisionTreeClassifier()
svc = SVC(probability=True)

voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr),
        ('dt', dt),
        ('svc', svc)
    ],
    voting='soft'
)

multi_voting_clf = MultiOutputClassifier(voting_clf)

param_grid = {
    'estimator__lr__C': [0.1, 10],
    'estimator__dt__max_depth': [5, 10],
    'estimator__svc__C': [0.1, 10],
    'estimator__svc__kernel': ['linear', 'rbf']
}

grid_search = GridSearchCV(estimator=multi_voting_clf, param_grid=param_grid, cv=3, scoring='f1_micro')
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

test_pred = best_model.predict(X_test)
train_pred = best_model.predict(X_train)


Train Accuracy: 0.5441378077129403
Test Accuracy: 0.5463647199046484


In [53]:

train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

Train Accuracy: 0.6538713715205341
Test Accuracy: 0.6324195470798569


In [54]:
print("Classification Metrics:")
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Precision (micro):", precision_score(y_test, test_pred, average='micro'))
print("Recall (micro):", recall_score(y_test, test_pred, average='micro'))
print("F1 Score (micro):", f1_score(y_test, test_pred, average='micro'))

y_prob = best_model.predict_proba(X_test)
y_prob = np.array([[cls[1] for cls in sample] for sample in y_prob])
y_prob = np.swapaxes(y_prob, 0, 1)

print("ROC AUC (macro):", roc_auc_score(y_test, y_prob, average='macro', multi_class='ovr'))
print("Log Loss:", log_loss(y_test, y_prob))
print("\nClassification Report:\n", classification_report(y_test, test_pred, zero_division=0))

print("\nConfusion Matrices (per class):")
for i in range(y_test.shape[1]):
    print(f"Class {i+1}:\n", confusion_matrix(y_test[:, i], test_pred[:, i]))

Classification Metrics:
Accuracy: 0.6324195470798569
Precision (micro): 0.8285714285714286
Recall (micro): 0.7490573152337858
F1 Score (micro): 0.7868105753044856
ROC AUC (macro): 0.9308588085175828
Log Loss: 0.712095017195726

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.85      0.82      1692
           1       0.91      0.80      0.85      1226
           2       0.86      0.72      0.79      1150
           3       0.80      0.67      0.73      1069
           4       0.56      0.04      0.08       122
           5       0.00      0.00      0.00        45

   micro avg       0.83      0.75      0.79      5304
   macro avg       0.65      0.51      0.54      5304
weighted avg       0.82      0.75      0.78      5304
 samples avg       0.81      0.78      0.78      5304


Confusion Matrices (per class):
Class 1:
 [[2103  400]
 [ 252 1440]]
Class 2:
 [[2868  101]
 [ 245  981]]
Class 3:
 [[2911  134]
 [ 317  833]]
Class 4

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [50]:
model_file_path = "/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/ensemble/votting_classifier_best.pkl"
with open(model_file_path, 'wb') as f:
    pickle.dump(best_model, f)


# Stacking Base

In [58]:
estimators = [
    ('rf', RandomForestClassifier(n_estimators=3, random_state=42)),
    ('svm', SVC(probability=True, kernel='rbf', C=1.0)),  # Probability=True for soft voting
    ('dt', DecisionTreeClassifier(max_depth=4))
]

meta_learner = LogisticRegression()

stacking_model = StackingClassifier(estimators=estimators, final_estimator=meta_learner)

multi_output_stacking_model = MultiOutputClassifier(stacking_model)

multi_output_stacking_model.fit(X_train, y_train)

train_pred = multi_output_stacking_model.predict(X_train)
test_pred = multi_output_stacking_model.predict(X_test)

train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

Train Accuracy: 0.6771770876795613
Test Accuracy: 0.6407628128724672


In [59]:
print("Classification Metrics:")
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Precision (micro):", precision_score(y_test, test_pred, average='micro'))
print("Recall (micro):", recall_score(y_test, test_pred, average='micro'))
print("F1 Score (micro):", f1_score(y_test, test_pred, average='micro'))

y_prob = multi_output_stacking_model.predict_proba(X_test)
y_prob = np.array([[cls[1] for cls in sample] for sample in y_prob])
y_prob = np.swapaxes(y_prob, 0, 1)

print("ROC AUC (macro):", roc_auc_score(y_test, y_prob, average='macro', multi_class='ovr'))
print("Log Loss:", log_loss(y_test, y_prob))
print("\nClassification Report:\n", classification_report(y_test, test_pred, zero_division=0))

print("\nConfusion Matrices (per class):")
for i in range(y_test.shape[1]):
    print(f"Class {i+1}:\n", confusion_matrix(y_test[:, i], test_pred[:, i]))

Classification Metrics:
Accuracy: 0.6407628128724672
Precision (micro): 0.8349676611725433
Recall (micro): 0.754524886877828
F1 Score (micro): 0.7927107061503417
ROC AUC (macro): 0.924911212122328
Log Loss: 0.7740407375208714

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.84      0.82      1692
           1       0.90      0.82      0.86      1226
           2       0.86      0.73      0.79      1150
           3       0.80      0.68      0.74      1069
           4       0.50      0.08      0.14       122
           5       1.00      0.04      0.09        45

   micro avg       0.83      0.75      0.79      5304
   macro avg       0.81      0.53      0.57      5304
weighted avg       0.83      0.75      0.78      5304
 samples avg       0.82      0.79      0.79      5304


Confusion Matrices (per class):
Class 1:
 [[2144  359]
 [ 272 1420]]
Class 2:
 [[2859  110]
 [ 225 1001]]
Class 3:
 [[2910  135]
 [ 311  839]]
Class 4:

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [60]:
model_file_path = "/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/ensemble/multi_output_stacking_model_base.pkl"
with open(model_file_path, 'wb') as f:
    pickle.dump(multi_output_stacking_model, f)


# CV


In [62]:
# prompt: take each saved model from the ensemble drive.. find its cross validation scores.. print it.. i dont want other performance metrics.. only cv... mean cv f1 cv

import pickle
import os
from sklearn.model_selection import cross_val_score

# Assuming your models are saved in the specified directory
ensemble_dir = "/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/ensemble/"

for filename in os.listdir(ensemble_dir):
    if filename.endswith(".pkl"):
        model_path = os.path.join(ensemble_dir, filename)
        print(f"Processing model: {filename}")

        try:
          with open(model_path, 'rb') as f:
              model = pickle.load(f)
          cv_scores = cross_val_score(model, X_train, y_train, cv=3, scoring='f1_micro')
          print(f"Cross-validation scores (f1_micro): {cv_scores}")
          print(f"Mean CV f1 score: {cv_scores.mean()}")
        except Exception as e:
          print(f"Error processing {filename}: {e}")
          continue


Processing model: tfidf_vectorizer.pkl
Error processing tfidf_vectorizer.pkl: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py", line 2074, in fit
    X = super().fit_transform(raw_documents)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/d

In [63]:
with open("/content/drive/Shareddrives/ML Sem 6 Assignment 1/nlp on research articles classification/ensemble/votting_classifier_base.pkl", 'rb') as f:
    model = pickle.load(f)
cv_scores = cross_val_score(model, X_train, y_train, cv=3, scoring='f1_micro')
print(f"Cross-validation scores (f1_micro): {cv_scores}")
print(f"Mean CV f1 score: {cv_scores.mean()}")

Cross-validation scores (f1_micro): [0.77079441 0.76696866 0.77421759]
Mean CV f1 score: 0.770660219164081
